## Demo of fmm Python API

In [1]:
from fmm import Network,NetworkGraph,FastMapMatch,FastMapMatchConfig,UBODT

### Load network data and graph

In [2]:
network = Network("../data/edges.shp")
print("Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count()))
graph = NetworkGraph(network)

Nodes 17 edges 30


### Precompute an UBODT file

**This step can be skipped if you already created one UBODT file.**

In [3]:
from fmm import UBODTGenAlgorithm

In [4]:
ubodt_gen = UBODTGenAlgorithm(network,graph)

In [5]:
status = ubodt_gen.generate_ubodt("../data/ubodt.txt", 4, binary=False, use_omp=True)
print(status)

Status: success
Time takes 0.01 seconds



### Load UBODT data

In [6]:
ubodt = UBODT.read_ubodt_csv("../data/ubodt.txt")

### Create FMM model

In [7]:
model = FastMapMatch(network,graph,ubodt)

### Define FMM configuration

In [8]:
k = 4
radius = 0.4
gps_error = 0.5
fmm_config = FastMapMatchConfig(k,radius,gps_error)

### Run map matching

#### Match a single trajectory

In [10]:
result = model.match_wkt(wkt,fmm_config)

In [11]:
print("Matched path: ", list(result.cpath))
print("Matched edge for each point: ", list(result.opath))
print("Matched edge index ",list(result.indices))
print("Matched geometry: ",result.mgeom.export_wkt())
print("Matched point ", result.pgeom.export_wkt())

Matched path:  [8, 11, 13, 18, 20, 24]
Matched edge for each point:  [8, 11, 18, 18, 20, 24]
Matched edge index  [0, 1, 3, 3, 4, 5]
Matched geometry:  LINESTRING(0.20081215 2,1 2,2 2,3 2,3 3,4 3,4 2.6233757)
Matched point  LINESTRING(0.20081215 2,1.4426201 2,3 2.1606638,3 2.7103814,3.7087218 3,4 2.6233757)


##### Print detailed match information

In [12]:
candidates = []
for c in result.candidates:
    candidates.append((c.edge_id,c.source,c.target,c.error,c.length,c.offset,c.spdist,c.ep,c.tp))

In [13]:
import pandas as pd
df = pd.DataFrame(candidates,
                  columns=["eid","source","target","error","length","offset","spdist","ep","tp"])

In [14]:
df.head()

eid  source  target     error  length    offset    spdist        ep  \
0    8       7       8  0.140890     1.0  0.200812  0.000000  0.961078   
1   11       8       5  0.148799     1.0  0.442620  1.241808  0.956684   
2   18       6      11  0.064089     1.0  0.160664  1.718044  0.991819   
3   18       6      11  0.064089     1.0  0.710381  0.549718  0.991819   
4   20      11      12  0.020692     1.0  0.708722  0.998340  0.999144   

         tp  
0  0.000000  
1  1.000000  
2  0.943813  
3  1.000000  
4  0.699640

#### Match trajectories in a GPS file

In [15]:
from fmm import GPSConfig,ResultConfig

In [16]:
input_config = GPSConfig()
input_config.file = "../data/trips.csv"
input_config.id = "id"

In [17]:
print(input_config.to_string())

gps file : ../data/trips.csv
id column : id
geom column : geom
timestamp column : timestamp
x column : x
y column : y
GPS point : false



In [18]:
result_config = ResultConfig()
result_config.file = "../data/mr.txt"
result_config.output_config.write_opath = True
print(result_config.to_string())

Result file : ../data/mr.txt
Output fields: opath cpath mgeom 


In [19]:
status = model.match_gps_file(input_config, result_config, fmm_config)

In [20]:
print(status)

Status: success
Time takes 0 seconds
Total points 17 matched 17
Total trajectories 3 matched 3
Map match percentage 1
Map match speed inf points/s 



In [23]:
help(Network.add_edge)

Help on function add_edge in module fmm:

add_edge(self, edge_id, source, target, geom)



In [27]:
import fmm

In [30]:
ls = fmm.LineString()

In [33]:
help(ls.add_point)

Help on method add_point in module fmm:

add_point(*args) method of fmm.LineString instance



In [34]:
ls.export_wkt()

'LINESTRING()'